<span style="color:red">Я загрузила бинарник мануально</span>, потому что текущие версии RusVectores вешают ZIP-архивы, с которыми мне изнутри Питона неудобно работать.
**Пункты 1 и 2**: Для начала импортируем необходимые библиотеки и построим векторную репрезентацию каждого слова. Поскольку пункт 2 требует от нас только сложить объекты array, этот шаг мы сделаем в той же ячейке.

In [2]:
# RUN THIS CELL!
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as hcluster
from scipy.spatial.distance import cosine
from sklearn.cluster import KMeans

In [2]:
# YOU DON'T HAVE TO RUN THIS CELL
# Я специально поместила этот импорт отдельно, потому как собираюсь залить свои данные
from gensim.models import KeyedVectors

def one_and_add():
    # Сначала сделаем список [глагол, имя, частотность]
    m = KeyedVectors.load_word2vec_format('model.bin', binary=True)
    # Я использую модель НКРЯ SkipGram 2019-го года
    with open('Leonova.txt', 'r', encoding='utf-8') as f:
        text = f.readlines()
    text = [i.replace('\t', '_').strip('\n').split('_') for i in text]
    # Теперь обработаем векторы
    verb = text[0][0] + '_VERB'
    verb_vec = m.get_vector(verb) # вектор глагола: один на всех
    c_text = []
    for i in text:
        noun = i[1] + '_NOUN'
        if noun in m:
            noun_vec = m.get_vector(noun) # вектор для каждого имени
            add_vec = verb_vec + noun_vec # вектор кумулятивного контекста
            c_text.append([i[0] + '_' + i[1], add_vec])
    # Делаем твёрдую копию, чтобы файлы не потерялись
    with open('clean_model.txt', 'w', encoding='utf-8') as f:
        for line in c_text:
            f.write(line[0] + '\t')
            for i in np.array(line[1]).tolist():
                f.write(str(i) + '\t')
            f.write('\n')
    print('File saved')

one_and_add()

File saved


**Пункт 3**: Теперь создадим из файла матрицу совместной встречаемости и попробуем с ней поработать. Начнём с иерархической кластеризации.

In [3]:
def matrix_rep():
    # Использую код из хэндаута со своей модификацией
    array = []
    d_mtr = {}
    with open('clean_model.txt', 'r', encoding='utf-8') as f_input:
        for line in f_input:
            # Очистим строку и создадим вектор
            line = line.strip().split('\t')
            vector = []
            for item in line[1:]:
                vector.append(float(item))
            array.append(vector)
            # Теперь сохраним элемент {фраза: массив}, она понадобится дальше
            d_mtr[line[0]] = vector 
    # Создаём матрицу
    X = np.array(array)
    return X, d_mtr


x, d_matrix = matrix_rep()

**Иерархическая кластеризация**: я ставлю threshold=1.154 и выбираю метод ward, потому что метод ward дал самые оптимально интерптерируемые результаты, а любое увеличение ведёт к единичному кластеру (плюс - этот threshold отражает визуальное распределение и приблизительно равен количеству значений в словарях).

Когда я делала это без смены методов, количество кластеров = 49, что заставляет меня думать, что, возможно, с моими данными что-то не так. Я проверила данные, двигаясь с самого начала: постепенно повышая порог частотности. К сожалению, количество кластеров начало быть похожим на какое-либо естественное распределение (то есть стало хотя бы меньше 20) только после установки порога частотности в 50, который оставляет всего 11 сочетаний для работы.

In [10]:
def hierarchical(X):
    # Использую код, представленный в хэндауте
    z = hcluster.linkage(X, method='ward')
    clusters = hcluster.fclusterdata(z, 1.15361)
    plt.figure()
    plt.show()
    print('Количество кластеров: %d' % len(set(clusters)))
    plt.figure()
    #plt.title('Иерархическая кластеризация для глагола закрыть')
    #dn = hcluster.dendrogram(z)
    plt.show()
    return clusters

hie_nabor = hierarchical(x)

<Figure size 432x288 with 0 Axes>

Количество кластеров: 1


<Figure size 432x288 with 0 Axes>

**К-средних**: я решила пойти типологическим путём и вычислить значение кластеров, используя количество значений для глагола закрыть в русскоязычных словарях и значение глаголы close в Merriam-Webster (в этом словаре значения уточняются до очень тонких оттенков, мне показалось, что это будет полезно).

Словарь Ожегова и Шведовой даёт 7 значений, из словаря Merriam-Webster я включила 1 значение (закрыть как в "закрыть небо"). Итого получился кластер из 8 значений.

In [17]:
import collections

k_nabor = KMeans(n_clusters=8).fit(x)
# чтобы оценить количество элементов в каждом кластере, воспользуемся Counter()
print(collections.Counter(k_nabor.labels_))

Counter({1: 46, 7: 17, 4: 10, 3: 8, 5: 6, 6: 6, 2: 6, 0: 2})


**Пункты 4 и 5** я снова делаю вместе, потому что у меня есть словарь, в котором хранятся фразы.

Прокомментирую не только внутрикодовыми комментариями, но и текстом: сначала я создаю словарь, в котором каждому значению кластера (т.е. номеру) соответствует список строк в нём (мне удобнее работать с таким объектом). Затем я вычисляю центр и по три элемента, стоящих близко к центру, причём оба метода (иерархический и к-средних) я обрабатываю в одной функции через параметр (опять же, вопрос моего удобства). Выходные данные - списки групп.

In [18]:
def comprehend(cluster):
    clus_dict = {}
    for j, c in enumerate(cluster):
        n = x[j]
        # Проверка на то, что объект уже в словаре
        try:
            value = clus_dict[str(c)]
        except KeyError:
            value = []
        # Добавление строки в словарь
        value.append(n)
        clus_dict[str(c)] = value
    return clus_dict


def cent_clus(clusters, kmeans=False):
    centers = []
    # Вне зависимости от типа объекта, создаем словарь {номер кластера: строки}
    if kmeans:
        clus_dict = comprehend(clusters.labels_)
        # Если объект kmeans, то высчитываем метрики центров
        centers = clusters.cluster_centers_
    else:
        clus_dict = comprehend(clusters)
    nabor = []
    for k in clus_dict.keys():
        try:
            value = clus_dict[k]
        except KeyError:
            continue # на случай капризного kmeans
        if len(value) <= 2:
            continue # игнорируем слишком маленькие
        value = np.array(value)
        if kmeans:
            # k-тый элемент - координата центра k-того кластера
            mean = centers[int(k)]
        else:
            mean = np.mean(value)
        # Высчитываем косинусное расстояние и берём только 3 ближайших
        cls = {cosine(mean, j): j for j in value}
        clst = sorted(cls, reverse=True)[:3]
        # Запаковываем в список [[первый, второй, третий], [...]]
        names = []
        for j in clst:
            for name, array in d_matrix.items():
                # Ищем нужную строку в {слово: строка}
                if np.all(cls[j] == array):
                    names.append(name)
        nabor.append(names)
    return nabor

# Создаём нужные нам списки
hie_nabor = cent_clus(hie_nabor)
k_nabor = cent_clus(k_nabor, kmeans=True)

# Выводим
print('Сортировка методом иерархической кластеризации: ')
for i in hie_nabor:
    print(', '.join(i))
print('\n')
print('Сортировка методом K-средних: ')
for i in k_nabor:
    print(', '.join(i))

Сортировка методом иерархической кластеризации: 
закрыть_губа, закрыть_глаз, закрыть_рот
закрыть_спектакль, закрыть_мгновение, закрыть_тема
закрыть_банк, закрыть_дыра, закрыть_труба
закрыть_туча, закрыть_облако, закрыть_небо
закрыть_рука, закрыть_колено, закрыть_нога


Сортировка методом K-средних: 
закрыть_губа, закрыть_глазок, закрыть_рот
закрыть_доступ, закрыть_позиция, закрыть_университет
закрыть_клапан, закрыть_кран, закрыть_занавес
закрыть_дыра, закрыть_луна, закрыть_солнце
закрыть_кабак, закрыть_кабинет, закрыть_магазин
закрыть_гроб, закрыть_стол, закрыть_постель
закрыть_банка, закрыть_тетрадь, закрыть_папка


**Пункт 6**

**Иерархическая кластеризация**: мне кажется, что этот метод сильнее зависит от выбранного имени для сочетания (в одну группу попадают явно разные значения _закрыть спектакль, закрыть театр, закрыть занавес_. Если говорить о иллюстративности, то этот метод мне кажется более когнитивно адекватным, чем k-средних: несмотря на наличие аутлаеров типа вышеуказанных, есть и очень точные наблюдения (например, специально мною выделенное значение закрыть_небо отражено отдельно, что мне представляется верным). В целом, как мне кажется, ему удалось охватить разные оттенки значений, хотя иногда его и "путала" принадлежность имени к определённому семантическому полю.

**K-средних**: я долго думала, что в моих данных что-то не так, потому что получилось, конечно, что-то очень странное. Мне не кажется, что этот метод как-либо репрезентативно описывает мои данные. Прогнав алгоритм через данные несколько раз, я так и не смогла найти оптимальную репрезентацию, которая бы делила значения на осмысленные группы